# Stage 3 - Enitity Matching Notebook
- Ankit Vij
- Amanpreet Singh Saini
- Joel Haynie

Date: 2018-04-18

Stages of our Pipeline: Clean -> Block -> Label -> ML'ing a Matcher

# Clean
We executed ```prepTableA.py``` & ```prepTableB.py```.

These take the raw two tables from Stage # 2 to a cleaned Table A & B

These go Column by Column and clean up and transform the data into a consistant and usable form.  Defaults were also defined.

 # Blocking
 We executed the ```blocker.py``` below is a summary.

In [1]:
import py_entitymatching as em
import os
import pandas as pd

In [12]:
# Load csv files as dataframes and set the key attribute in the dataframe
path_A = '..'+ os.sep + 'data' + os.sep + 'A.csv'
A = em.read_csv_metadata(path_A, key='ID')
path_B = '..'+ os.sep + 'data' + os.sep + 'B.csv'
B = em.read_csv_metadata(path_B, key='ID')

Metadata file is not present in the given path; proceeding to read the csv file.
Metadata file is not present in the given path; proceeding to read the csv file.


In [20]:
#Number of Tuples in A
print("A Tuples: "+str(len(A)))
A.head(3)

A Tuples: 3250


,ID,Title,Score,Rating,Genre,Directed By,Written By,Box Office,Release Date,Runtime,Studio
0,A_0001,Avengers: Infinity War,0,NOT RATED,Fantasy,Anthony Russo,Christopher Markus,0,2018-04-27,156,Marvel Studios
1,A_0002,Tomb Raider,69,PG-13,Action,Roar Uthaug,Geneva Robertson-Dworet,31318108,2018-03-16,118,GK Films
2,A_0003,Black Panther,78,PG-13,Sci-Fi,Ryan Coogler,Ryan Coogler,614258236,2018-02-16,134,Marvel Studios


In [21]:
#Number of Tuples in A
print("A Tuples: "+str(len(B)))
B.head(3)

A Tuples: 3005


,ID,Title,Score,Rating,Genre,Directed By,Written By,Box Office,Release Date,Runtime,Studio
0,B_0001,Justice League,40,PG-13,Drama,Zack Snyder,Chris Terrio,227032490,2017-11-17,110,Warner Bros. Pictures
1,B_0002,Ferdinand,71,PG,Drama,Carlos Saldanha,Robert L. Baird,70466891,2017-12-15,0,NaN
2,B_0003,Court,100,NOT RATED,Drama,Chaitanya Tamhane,Chaitanya Tamhane,0,2015-07-15,116,Zeitgeist Films


Gernerate 3 candidate sets:
1. AttrEquivalenceBlocker on the movie 'Release Date'
1. OverlapBlocker on the movie 'Title'
1. AttrEquivalenceBlocker on the movie 'Title'.

These are all Unioned together into set D
Finally a Rule Based Blocker over the Levenshtein distance for the 'Title' < 0.4


In [28]:
#1 
ab = em.AttrEquivalenceBlocker()
# Block tables using 'year' attribute : same year include in candidate set
C1 = ab.block_tables(A, B, 'Release Date', 'Release Date',
                     l_output_attrs=['Title', 'Genre', 'Score', 'Release Date', 'Rating', 'Directed By', 'Written By', 'Studio'],
                     r_output_attrs=['Title', 'Genre', 'Score', 'Release Date', 'Rating', 'Directed By', 'Written By', 'Studio'])
print("C1 Set Size: ", len(C1))

C1 Set Size:  191237


In [29]:
#2
ob = em.OverlapBlocker()
# Block over title attribute
C2 = ob.block_tables(A, B, 'Title', 'Title', show_progress=False, overlap_size=2, rem_stop_words=True,
                     l_output_attrs=['Title', 'Genre', 'Score', 'Release Date', 'Rating', 'Directed By', 'Written By','Studio'],
                     r_output_attrs=['Title', 'Genre', 'Score', 'Release Date', 'Rating', 'Directed By', 'Written By','Studio'])
print("C2 Set Size: ", len(C2))

C2 Set Size:  1826


In [30]:
#3
# Attribute Equivalence Blocker for Title
C3 = ab.block_tables(A, B, 'Title', 'Title',
                     l_output_attrs=['Title', 'Genre', 'Score', 'Release Date', 'Rating', 'Directed By', 'Written By','Studio'],
                     r_output_attrs=['Title', 'Genre', 'Score', 'Release Date', 'Rating', 'Directed By', 'Written By','Studio'])
print("C3 Set Size: ", len(C3))

C3 Set Size:  607


In [31]:
#Union the last three
D = em.combine_blocker_outputs_via_union([C1, C2, C3])
print("D Set Size: ", len(D))

D Set Size:  192807


In [32]:
# Rule based blocker on D
block_f = em.get_features_for_blocking(A, B, validate_inferred_attr_types=False)
rb = em.RuleBasedBlocker()
rb.add_rule(['Title_Title_lev_sim(ltuple, rtuple) < 0.4'], block_f)
C = rb.block_candset(D, show_progress=False)
print("C Set Size: ", len(C))

C Set Size:  3335


We Saved off C set to a csv file in the data directory.

We did a bit of debugging with ```em.debug_blocker(C, A, B, output_size=200)```

We generated our set S with ```S = em.sample_table(C, 300)```

We Then labeled our data with the ```G = em.label_table(S, 'label')```

And finally saved G set to a csv file in the data directory.

# ML'ing a Matcher
We executed ```createClassifier.py``` below is a summary.

In [35]:
path_G = '..' + os.sep + 'data' + os.sep + 'G.csv'
G = em.read_csv_metadata(path_G, key='_id', ltable=A, rtable=B, fk_ltable='ltable_ID', fk_rtable='rtable_ID')
print("G Tuples: "+str(len(G)))

Metadata file is not present in the given path; proceeding to read the csv file.


G Tuples: 300


In [37]:
# Create our Sets I & J
IJ = em.split_train_test(G, train_proportion=0.7, random_state=0)
I = IJ['train']
J = IJ['test']
print("I Tuples: "+str(len(I)))
print("J Tuples: "+str(len(J)))

I Tuples: 210
J Tuples: 90


In [38]:
# Obtain our feature sets
feature_table = em.get_features_for_matching(A, B, validate_inferred_attr_types=False)

# extract our feature vectors from I
H = em.extract_feature_vecs(I, feature_table=feature_table, attrs_after='label', show_progress=False)

# Clean up some missing data.
H.fillna(value=0, inplace=True)

We Saved off I & J Sets to a csv file in the data directory

We ran through the following classifers using(```em.select_matcher```) with precision and recall on set H (from I):
1. ```em.DTMatcher(name='DecisionTree', random_state=0)```
1. ```em.SVMMatcher(name='SVM', random_state=0)```
1. ```em.RFMatcher(name='RF', random_state=0)```
1. ```em.LogRegMatcher(name='LogReg', random_state=0)```
1. ```em.LinRegMatcher(name='LinReg')```
1. ```em.NBMatcher(name='NaiveBayes')```

We will only show one in this notebook. Choose **Random Forest**

In [39]:
# Define our matcher, this was done forall above.
rf = em.RFMatcher(name='RF', random_state=0)

# Build up or test set feature vectors from J
L = em.extract_feature_vecs(J, feature_table=feature_table, attrs_after='label', show_progress=False)

# Clean up some missing data.
L.fillna(value=0, inplace=True)

#Fit to our set Development Set H (from I)
rf.fit(table=H, exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'label'], target_attr='label')

# Use our test set L (from J) to measure the Matchers effectiveness
predictions = rf.predict(table=L, exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'label'],
                         append=True, target_attr='predicted', inplace=False)

# How did it do?
eval_result = em.eval_matches(predictions, 'label', 'predicted')
print('\n Random Forest Result-')
em.print_eval_summary(eval_result)


 Random Forest Result-
Precision : 100.0% (22/22)
Recall : 100.0% (22/22)
F1 : 100.0%
False positives : 0 (out of 22 positive predictions)
False negatives : 0 (out of 68 negative predictions)


**Fin!**